In [60]:
cd '~/cloudFinder/'

/home/antoine/cloudFinder


In [76]:
import jupyterlab_dash
import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output
import feather as ft 
import pandas as pd
import base64
import dash_table
from io import BytesIO
from skimage import measure


import plotly.graph_objects as go
from src.image.utils import load_image
from src.image.transform import resize_image
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
viewer = jupyterlab_dash.AppViewer(port=9996)

In [77]:
COMPUTE_PERFORMANCE = False

In [78]:
df_cv = ft.read_dataframe('data/process/sky_segmentation/model_1_cv.ft')
df_cv.head()

,img_name,is_sky,is_sky_PREDICTED,fold,is_sky_PREDICTED_COR
0,labelme_aacpgupgzvdjapw.jpg,0.0,0.0,3,0.0
1,labelme_aacpgupgzvdjapw.jpg,0.0,0.0,3,0.0
2,labelme_aacpgupgzvdjapw.jpg,0.0,0.0,3,0.0
3,labelme_aacpgupgzvdjapw.jpg,0.0,0.0,3,0.0
4,labelme_aacpgupgzvdjapw.jpg,0.0,0.0,3,0.0


In [79]:
df_meta = ft.read_dataframe('data/process/sky_segmentation/features_meta.ft')
df_meta.head()

,img_name,img_path,size_h,size_l,sky_coverage,sky_coverage_after_resize,is_used
0,labelme_aacpgupgzvdjapw.jpg,http://labelme.csail.mit.edu/Images/users/anto...,100,133,0.213749,0.216015,True
1,labelme_aajuldidvlcyzhv.jpg,http://labelme.csail.mit.edu/Images/users/anto...,100,133,0.102192,0.103534,True
2,labelme_aajvoauwthyaehf.jpg,http://labelme.csail.mit.edu/Images/users/anto...,0,0,0.002079,0.000000,False
3,labelme_aalbzqrimafwbiv.jpg,http://labelme.csail.mit.edu/Images/users/anto...,100,133,0.335283,0.337744,True
4,labelme_aatpnkdmtvmagyh.jpg,http://labelme.csail.mit.edu/Images/users/anto...,0,0,0.022681,0.000000,False


In [80]:
if COMPUTE_PERFORMANCE:
    score = []

    for i in ["is_sky_PREDICTED", "is_sky_PREDICTED_COR"]:
        tmp = {
            "accuracy": df_cv.groupby("img_name").apply(
                lambda x: accuracy_score(x["is_sky"], x[i])
            ),
            "recall": df_cv.groupby("img_name").apply(
                lambda x: recall_score(x["is_sky"], x[i])
            ),
            "precision": df_cv.groupby("img_name").apply(
                lambda x: precision_score(x["is_sky"], x[i])
            ),
            "fold": df_cv.groupby("img_name").apply(lambda x: list(set(x["fold"]))[0]),
        }
        tmp_df = pd.concat(tmp.values(), axis=1)
        tmp_df.columns = tmp.keys()
        tmp_df["prediction_label"] = i

        score.append(tmp_df)

    score = pd.concat(score)
    score.reset_index(inplace=True, drop=False)

    ft.write_dataframe(score, "data/process/sky_segmentation/score.ft")
else:
    score = ft.read_dataframe("data/process/sky_segmentation/score.ft")

score.head()

,img_name,accuracy,recall,precision,fold,prediction_label
0,labelme_aacpgupgzvdjapw.jpg,0.870376,0.588235,0.757508,3,is_sky_PREDICTED
1,labelme_aajuldidvlcyzhv.jpg,0.928571,0.851126,0.611372,5,is_sky_PREDICTED
2,labelme_aalbzqrimafwbiv.jpg,0.985489,0.969501,0.987304,4,is_sky_PREDICTED
3,labelme_aaybneesrunupnk.jpg,0.936241,0.922240,0.832164,1,is_sky_PREDICTED
4,labelme_abxjjdteigmqnyg.jpg,0.870526,0.794927,1.000000,3,is_sky_PREDICTED


In [81]:
all_img = set(df_cv['img_name'])
all_metric = set(['accuracy', 'recall', 'precision'])

In [82]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.Div([html.H1(children="Sky segmentation"),], className="row"),
        html.Div(
            [
                html.Div(
                    [
                        html.H6(children="Metric"),
                        dcc.Dropdown(
                            options=[{"label": i, "value": i} for i in all_metric],
                            value="accuracy",
                            multi=False,
                            id="metric",
                        ),
                        html.H6(children="Image score"),
                        dash_table.DataTable(
                            id="score_table",
                            columns=[
                                {"name": i, "id": i}
                                for i in ["Image", "Score", "Corrected Score"]
                            ],
                            sort_action="native",
                            page_size=10,
                            style_table={"maxHeight": "50", "overflowY": "scroll"},
                        ),
                    ],
                    className="four columns",
                ),
                html.Div(
                    [
                        html.H6(children="Score"),
                        dcc.Graph(id="box_plot"),
                    ],
                    className="eight columns",
                ),
            ],
            className="row",
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.H6(children="image name"),
                        dcc.Dropdown(
                            options=[{"label": i, "value": i} for i in all_img],
                            value="labelme_aacpgupgzvdjapw.jpg",
                            multi=False,
                            id="img_name",
                        ),
                        html.H6(children="With correction"),
                        dcc.RadioItems(
                            id="corr",
                            options=[
                                {"label": i, "value": i} for i in ["True", "False"]
                            ],
                            value="False",
                            labelStyle={"display": "inline-block"},
                        ),
                        html.H6(children="confusion matrix"),
                        dcc.Markdown(
                            children="""
                                            |          | pos      | neg      |
                                            |----------|----------|----------|
                                            | Pred Pos | 0   | 0  | 
                                            | Pred Neg | 0   | 0  | 
                                        """,
                            id="confusion_matrix",
                        ),
                    ],
                    className="four columns",
                ),
                html.Div(
                    [
                        html.H6(children="Image"),
                        html.Img(src="", id="img_plot"),
                    ],
                    className="eight columns",
                ),
            ],
            className="row",
        ),
    ],
    className="ten columns offset-by-one",
)

In [83]:
@app.callback(
    Output("box_plot", "figure"), [Input("metric", "value")]
)
def update_plot(metric):
    data = []
    for fold_label, fold_df in score.groupby("fold"):
        data.append(go.Box(y=fold_df[fold_df.prediction_label == "is_sky_PREDICTED"][metric], name=str(fold_label) + "_without_cor"))
        data.append(go.Box(y=fold_df[fold_df.prediction_label == "is_sky_PREDICTED_COR"][metric], name=str(fold_label) + "_with_cor"))
    return {"data": data}

In [84]:
@app.callback(
    Output("confusion_matrix", "children"),
    [Input("img_name", "value"), Input("corr", "value")],
)
def update_confusion_matrix(img_name, with_correction):

    if with_correction == "True":
        pred_label = "is_sky_PREDICTED_COR"
    else:
        pred_label = "is_sky_PREDICTED"

    tmp = df_cv[df_cv["img_name"] == img_name].copy()

    tn, fp, fn, tp = confusion_matrix(tmp["is_sky"], tmp[pred_label]).ravel()

    txt = """
        |          | pos      | neg      |
        |----------|----------|----------|
        | Pred Pos | {}   | {}  | 
        | Pred Neg | {}   | {}    |
        """.format(
        tp, fp, fn, tn
    )
    return txt

In [85]:
@app.callback(Output('score_table', 'data'), [Input('metric', 'value')])
def update_table(metric):
    tmp = score[['img_name', metric, "prediction_label"]].groupby(['img_name', 'prediction_label']).mean().unstack(level=1).reset_index(drop=False)
    tmp.columns = ["Image", "Score", "Corrected Score"]
    return tmp.to_dict("rows")


In [86]:
@app.callback(
    Output("img_plot", "src"), [Input("img_name", "value"), Input("corr", "value")]
)
def update_image(img_name, with_correction):

    if with_correction == "True":
        pred_label = "is_sky_PREDICTED_COR"
    else:
        pred_label = "is_sky_PREDICTED"

    size_h, size_l = df_meta[df_meta.img_name == img_name][["size_h", "size_l"]].iloc[0]

    df_cv_image = df_cv[df_cv.img_name == img_name].copy()

    sky = df_cv_image["is_sky"].values.reshape((size_h, size_l))
    sky_contours = measure.find_contours(sky, 0)
    
    sky_predicted = df_cv_image[pred_label].values.reshape((size_h, size_l))
    sky_predicted_contours = measure.find_contours(sky_predicted, 0)

    img = load_image("data/img/sky_segmentation/", img_name)
    img_resized = resize_image(img, 100)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img_resized)
    for n, c in enumerate(sky_contours):
        ax.plot(c[:, 1], c[:, 0], linewidth=3, color="r")
    for n, c in enumerate(sky_predicted_contours):
        ax.plot(c[:, 1], c[:, 0], linewidth=3, color="y")
    figfile = BytesIO()
    plt.savefig(figfile, format="png")
    figdata_png = base64.b64encode(figfile.getvalue()).decode()

    src = "data:image/png;base64,{}".format(figdata_png)

    return src

In [87]:
viewer.stderr_queue.write('Running on'.encode('utf-8'))
viewer.show(app)

In [75]:
viewer.terminate()

In [ ]:
img_name = 'labelme_aaybneesrunupnk.jpg'
img = load_image('data/img/sky_segmentation/', img_name)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img)

In [ ]:
size_h, size_l = df_meta[df_meta.img_name == img_name][['size_h', 'size_l']].iloc[0]

In [ ]:
img_resized = resize_image(img, 100)

In [ ]:
sky = df_cv[df_cv.img_name==img_name]['is_sky'].values.reshape((size_h, size_l))
sky_contours = measure.find_contours(sky,0)
sky_predicted = df_cv[df_cv.img_name==img_name]['is_sky_PREDICTED'].values.reshape((size_h, size_l))
sky_predicted_contours = measure.find_contours(sky_predicted, 0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img_resized)

for n, c in enumerate(sky_contours):
    ax.plot(c[:, 1], c[:, 0], linewidth=3, color="r")
for n, c in enumerate(sky_predicted_contours):
    ax.plot(c[:, 1], c[:, 0], linewidth=3, color="y")

In [ ]:
sky

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(sky)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(sky_predicted)